In [12]:
from collections import defaultdict 
import sys

In [13]:
class Heap(): 
  
    def __init__(self): 
        self.array = [] 
        self.size = 0
        self.pos = [] 
  
    def newMinHeapNode(self, v, dist): 
        minHeapNode = [v, dist] 
        return minHeapNode 
  
    # A utility function to swap two nodes  
    # of min heap. Needed for min heapify 
    def swapMinHeapNode(self,a, b): 
        t = self.array[a] 
        self.array[a] = self.array[b] 
        self.array[b] = t 
  
    # A standard function to heapify at given idx 
    # This function also updates position of nodes  
    # when they are swapped.Position is needed  
    # for decreaseKey() 
    def minHeapify(self, idx): 
        smallest = idx 
        left = 2*idx + 1
        right = 2*idx + 2
  
        if (left < self.size and self.array[left][1] < self.array[smallest][1]): 
            smallest = left 
  
        if (right < self.size and self.array[right][1] < self.array[smallest][1]): 
            smallest = right 
  
        # The nodes to be swapped in min  
        # heap if idx is not smallest 
        if (smallest != idx): 
  
            # Swap positions 
            self.pos[ self.array[smallest][0] ] = idx 
            self.pos[ self.array[idx][0] ] = smallest 
  
            # Swap nodes 
            self.swapMinHeapNode(smallest, idx) 
  
            self.minHeapify(smallest) 
  
    # Standard function to extract minimum  
    # node from heap 
    def extractMin(self): 
  
        # Return NULL wif heap is empty 
        if self.isEmpty() == True: 
            return
  
        # Store the root node 
        root = self.array[0] 
  
        # Replace root node with last node 
        lastNode = self.array[self.size - 1] 
        self.array[0] = lastNode 
  
        # Update position of last node 
        self.pos[lastNode[0]] = 0
        self.pos[root[0]] = self.size - 1
  
        # Reduce heap size and heapify root 
        self.size -= 1
        self.minHeapify(0) 
  
        return root 
  
    def isEmpty(self): 
        return True if self.size == 0 else False
  
    def decreaseKey(self, v, dist): 
  
        # Get the index of v in  heap array 
  
        i = self.pos[v] 
  
        # Get the node and update its dist value 
        self.array[i][1] = dist 
  
        # Travel up while the complete tree is  
        # not hepified. This is a O(Logn) loop 
        while (i > 0 and self.array[i][1] < self.array[(i - 1) // 2][1]): 
  
            # Swap this node with its parent 
            self.pos[ self.array[i][0] ] = (i-1)/2
            self.pos[ self.array[(i-1)//2][0] ] = i 
            self.swapMinHeapNode(i, (i - 1)//2 ) 
  
            # move to parent index 
            i = (i - 1) // 2; 
  
    # A utility function to check if a given  
    # vertex 'v' is in min heap or not 
    def isInMinHeap(self, v):
        if (self.pos[v] < self.size): 
            return True
        return False

In [14]:

class FibonacciHeap:

    # internal node class
    class Node:
        def __init__(self, key, value):
            self.key = key
            self.value = value
            self.parent = self.child = self.left = self.right = None
            self.degree = 0
            self.mark = False

    # function to iterate through a doubly linked list
    def iterate(self, head):
        node = stop = head
        flag = False
        while True:
            if node == stop and flag is True:
                break
            elif node == stop:
                flag = True
            yield node
            node = node.right

    # pointer to the head and minimum node in the root list
    root_list, min_node = None, None

    # maintain total node count in full fibonacci heap
    total_nodes = 0

    # return min node in O(1) time
    def find_min(self):
        return self.min_node

    # extract (delete) the min node from the heap in O(log n) time
    # amortized cost analysis can be found here (http://bit.ly/1ow1Clm)
    def extract_min(self):
        z = self.min_node
        if z is not None:
            if z.child is not None:
                # attach child nodes to root list
                children = [x for x in self.iterate(z.child)]
                for i in range(0, len(children)):
                    self.merge_with_root_list(children[i])
                    children[i].parent = None
            self.remove_from_root_list(z)
            # set new min node in heap
            if z == z.right:
                self.min_node = self.root_list = None
            else:
                self.min_node = z.right
                self.consolidate()
            self.total_nodes -= 1
        return z

    # insert new node into the unordered root list in O(1) time
    # returns the node so that it can be used for decrease_key later
    def insert(self, key, value=None):
        n = self.Node(key, value)
        n.left = n.right = n
        self.merge_with_root_list(n)
        if self.min_node is None or n.key < self.min_node.key:
            self.min_node = n
        self.total_nodes += 1
        return n

    # modify the key of some node in the heap in O(1) time
    def decrease_key(self, x, k):
        if k > x.key:
            return None
        x.key = k
        y = x.parent
        if y is not None and x.key < y.key:
            self.cut(x, y)
            self.cascading_cut(y)
        if x.key < self.min_node.key:
            self.min_node = x

    # merge two fibonacci heaps in O(1) time by concatenating the root lists
    # the root of the new root list becomes equal to the first list and the second
    # list is simply appended to the end (then the proper min node is determined)
    def merge(self, h2):
        H = FibonacciHeap()
        H.root_list, H.min_node = self.root_list, self.min_node
        # fix pointers when merging the two heaps
        last = h2.root_list.left
        h2.root_list.left = H.root_list.left
        H.root_list.left.right = h2.root_list
        H.root_list.left = last
        H.root_list.left.right = H.root_list
        # update min node if needed
        if h2.min_node.key < H.min_node.key:
            H.min_node = h2.min_node
        # update total nodes
        H.total_nodes = self.total_nodes + h2.total_nodes
        return H

    # if a child node becomes smaller than its parent node we
    # cut this child node off and bring it up to the root list
    def cut(self, x, y):
        self.remove_from_child_list(y, x)
        y.degree -= 1
        self.merge_with_root_list(x)
        x.parent = None
        x.mark = False

    # cascading cut of parent node to obtain good time bounds
    def cascading_cut(self, y):
        z = y.parent
        if z is not None:
            if y.mark is False:
                y.mark = True
            else:
                self.cut(y, z)
                self.cascading_cut(z)

    # combine root nodes of equal degree to consolidate the heap
    # by creating a list of unordered binomial trees
    def consolidate(self):
        A = [None] * self.total_nodes
        nodes = [w for w in self.iterate(self.root_list)]
        for w in range(0, len(nodes)):
            x = nodes[w]
            d = x.degree
            while A[d] != None:
                y = A[d]
                if x.key > y.key:
                    temp = x
                    x, y = y, temp
                self.heap_link(y, x)
                A[d] = None
                d += 1
            A[d] = x
        # find new min node - no need to reconstruct new root list below
        # because root list was iteratively changing as we were moving
        # nodes around in the above loop
        for i in range(0, len(A)):
            if A[i] is not None:
                if A[i].key < self.min_node.key:
                    self.min_node = A[i]

    # actual linking of one node to another in the root list
    # while also updating the child linked list
    def heap_link(self, y, x):
        self.remove_from_root_list(y)
        y.left = y.right = y
        self.merge_with_child_list(x, y)
        x.degree += 1
        y.parent = x
        y.mark = False

    # merge a node with the doubly linked root list
    def merge_with_root_list(self, node):
        if self.root_list is None:
            self.root_list = node
        else:
            node.right = self.root_list.right
            node.left = self.root_list
            self.root_list.right.left = node
            self.root_list.right = node

    # merge a node with the doubly linked child list of a root node
    def merge_with_child_list(self, parent, node):
        if parent.child is None:
            parent.child = node
        else:
            node.right = parent.child.right
            node.left = parent.child
            parent.child.right.left = node
            parent.child.right = node

    # remove a node from the doubly linked root list
    def remove_from_root_list(self, node):
        if node == self.root_list:
            self.root_list = node.right
        node.left.right = node.right
        node.right.left = node.left

    # remove a node from the doubly linked child list
    def remove_from_child_list(self, parent, node):
        if parent.child == parent.child.right:
            parent.child = None
        elif parent.child == node:
            parent.child = node.right
            node.right.parent = parent
        node.left.right = node.right
        node.right.left = node.left

In [15]:
class Graph(): 
  
    def __init__(self, V): 
        self.V = V 
        self.graph = defaultdict(list)
      
    
    def vertices(self):
        return list(self.graph)
    
    def edges(self):
        return self.graph
    
    def addEdge(self, src, dest, weight):
        newNode = [dest, weight] 
        self.graph[src].insert(0, newNode) 


In [16]:
def getPath(prev,s):
    path = [[] for i in range(len(prev))]   
    for i in range(len(prev)):
        path[i].append(i)
        curr = prev[i]
        while True:
            if(curr == s):
                break
            path[i].append(curr)
            if(curr != None):
                curr = prev[curr]
            else:
                break
    return path

def printPath(src,dist,prev):
    paths = getPath(prev,src)
    
    for i in range(0,len(dist)):
        if(dist[i] < sys.maxsize):
            print("custo %d " % dist[i],end="")
            for j in paths[i]:
                print("v%d " % j, end="")
            if (i != 0):
                print("v0 %d" % i)
            else:
                print("%d" % i)
        else:
            print("custo Inf v%d %d" % (i,i))

In [17]:
def dijkstra(G,s):
        V = G.V
        dist = []
        minHeap = Heap() 
        prev = []
        
        for v in range(V):
            dist.append(sys.maxsize)
            prev.append(None)
            minHeap.array.append( minHeap.newMinHeapNode(v, dist[v]) ) 
            minHeap.pos.append(v) 

        minHeap.pos[s] = s
        dist[s] = 0
        prev[s] = s
        minHeap.decreaseKey(s, dist[s])
        
        minHeap.size = V;
        
        while minHeap.isEmpty() == False:
            newHeapNode = minHeap.extractMin()  
            u = newHeapNode[0]
            for pCrawl in G.graph[u]: 
                v = pCrawl[0]
                if (minHeap.isInMinHeap(v) and dist[u] != sys.maxsize and pCrawl[1] + dist[u] < dist[v]): 
                        dist[v] = pCrawl[1] + dist[u]
                        minHeap.decreaseKey(v, dist[v]) 
                        prev[v]= u 
                        
        return dist,prev

In [18]:
def DijkstraFibHeap(G,s):
    V = G.V
    visited = [False]*V
    dist = [float(sys.maxsize)] * V
    prev = [None]*V
    heapNodes = [None]*V
    
    
    heap = FibonacciHeap()
    for i in range(V):
        heapNodes[i] = heap.insert(float('inf'), i)
    
    dist[s] = 0
    prev[s] = s
    heap.decrease_key(heapNodes[s], 0)
    
    while heap.total_nodes:
        current = heap.extract_min().value
        u = current
        visited[current] = True
        for pCrawl in G.graph[u]: 
                v = pCrawl[0]
                if (not visited[v]) :
                    if (dist[u] != sys.maxsize and pCrawl[1] + dist[u] < dist[v]): 
                            dist[v] = pCrawl[1] + dist[u]
                            heap.decrease_key(heapNodes[v], dist[v]) 
                            prev[v]= u
    
    return dist,prev

In [19]:
def BellmanFord(G, s):
        dist = [sys.maxsize] * G.V
        prev = [None] * G.V
        dist[s] = 0
        prev[s] = s
        
        for l in range(G.V -1):
            for i,j in G.edges().items():
                for k in j:
                    u = i
                    v,w = k
                    if dist[u] != float("Inf") and dist[u] + w < dist[v]: 
                            dist[v] = dist[u] + w
                            prev[v] = u
                            
        for i,j in G.edges().items():
                for k in j:
                    u = i
                    v,w = k
                    if dist[u] != float("Inf") and dist[u] + w < dist[v]: 
                        print ("Graph contains negative weight cycle")
                        return
        return dist,prev

In [30]:
def FloydWarshall(G): 
    V = G.V
    dist = [[sys.maxsize] * V for i in range(V)]
    prox  = [[0] * V for i in range(V)] 
    
    
    for i in range(V):
        dist[i][i] = 0
        
    for i,j in G.edges().items():
                for k in j:
                    u = i
                    v,w = k
                dist[u-1][v-1] = w
                prox[u-1][v-1] = v-1
    
        
    for k in range(V):
        for i in range(V):
            for j in range(V):
                if (dist[i][j] > dist[i][k] + dist[k][j]):
                    dist[i][j] = dist[i][k]+ dist[k][j]
                    prox[i][k] = prox[i][j]
                    
    for i in range(V):
        for j in range(V):
            if i != j:
                path = [i]
                
                while path[-1] != j:
                    print(path,j)
                    path.append(prox[path[-1]][j])
                    print(path)
                print("%d → %d  %4d %s" % (i + 1, j + 1, dist[i][j],' → '.join(str(p + 1) for p in path)))
                
    return dist,prox

def Floyd_Path(prox,v, u):
    if prox[u][v] == None:
        return []
    path = [u]
    while u != v:
        print(u,v)
        u = prox[u][v]
        path.append(u)
    return path

In [28]:
#filename = "/home/rkanehisa/Workspace/grafos-t1-mst/mst/dataset/cormen_in.txt"
#a = open(filename,'r').read().split('\n')

#G = Graph(int(a[0]))

#for i in range(2,len(a)):
#    tmp = a[i].split(" ")
#    G.addEdge(int(tmp[0]),int(tmp[1]),float(tmp[2]))

a = [[1, 3, -2], [2, 1, 4], [2, 3, 3], [3, 4, 2], [4, 2, -1]]
G = Graph(4)
for i in a:
    G.addEdge(i[0],i[1],i[2])

In [35]:
#dist,prox = FloydWarshall(G)
list(G.edges())


[1, 2, 3, 4]

In [ ]:
src = 0
dist,path = dijkstra(G,src)
printPath(src,dist,path)

In [ ]:
src = 0
dist,path = DijkstraFibHeap(G,src)
printPath(src,dist,path)

In [ ]:
out = BellmanFord(G,src)
if(out != None):
    dist,path = out
    printPath(src,dist,path)
else:
    print("Ciclo negativo")